# DenseNet Model Implementation

In [4]:
import os
import glob
import cv2
#import math
import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
#from torch.utils.data import Subset
from torch.utils.data import random_split
import torch.nn as nn
import torch.optim as optim
import torchvision
#from torchvision import transforms
#import sklearn
#from PIL import Image
#import zipfile
#import urllib.request
import os.path
#from IPython.display import display
#from helperFunctions import create_image_stack
from helperFunctions import tile
#from helperFunctions import create_classEncoding
from analysisFunctions import compute_score_with_logits
from dataClasses import DataPreprocessing
from dataClasses import DenseNet121
import dotenv
dotenv.load_dotenv()

True

In [5]:
 # Check if PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using device: {device}")

# Data Preprocessing 
data = DataPreprocessing()

testOnly = eval(os.getenv("testOnly"))
if testOnly:
    trainSize = 0
    testSize = len(data) 
else:
    trainSize = len(data) 
    testSize = 0

train_set, test_set = random_split(data, [trainSize, testSize])

if testOnly:
    testloader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False)
else: 
    trainloader = torch.utils.data.DataLoader(train_set, batch_size=2, shuffle=False, num_workers=0)

if device == "mps":
    model = DenseNet121().to(device)
    model = nn.DataParallel(model).to(device)
elif device == "cuda":
    model = DenseNet121().cuda()
    model = nn.DataParallel(model).cuda()
else :
    model = DenseNet121()
    model = nn.DataParallel(model)

Using device: cuda


In [6]:

%%time

#parameters to save/load models
testOnly = os.getenv('testOnly')
if testOnly == 'True':
    saveModel = 'False'
else:
    saveModel = os.getenv('saveModel')

modelPath = os.getenv('SAVE_LOAD_MODEL_PATH')
downImageSize = os.getenv('downImageSize')
nClasses = os.getenv('nClasses')
modelFname = "dense" + str(nClasses) + "class" + str(downImageSize) + "pix.pt"
modelPathFile = modelPath + modelFname

if testOnly=='False': #We are doing full training 

    #INITIAL GUESS FROM PREVIOUSLY TRAINED MODEL
    if os.path.isfile(modelPathFile):
        model = torch.load(modelPathFile)
    else:
        pass 

    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # RMSprop, Adam

    nEpochs = eval(os.getenv("nEpochs"))
    useImLevels = eval(os.getenv("useImLevels"))

    for epoch in range(nEpochs): #loop over the dataset multiple times

        running_loss = 0.0
        correct = 0
        total = 0 
        for i, (images, labels, image_names) in enumerate(trainloader, 0): # get the inputs; data is a list of [images, labels]

            # zero the parameter gradients
            optimizer.zero_grad()
            
            if device == "mps":
                images = images.to(device)
                labels = tile(labels, 0, useImLevels).to(device) #duplicate for each crop the label 
            elif device == "cuda":
                images = images.cuda()
                labels = tile(labels, 0, useImLevels).cuda()
            else:
                labels = tile(labels, 0, useImLevels)
        
            #format input
            n_batches, n_crops, channels, height, width = images.size()
            image_batch = torch.autograd.Variable(images.view(-1, channels, height, width)) 
            
            # forward + backward + optimize
            outputs = model(image_batch)
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            correct += compute_score_with_logits(outputs, labels, device).sum()
            total += labels.size(0)

        print('Epoch: %d, loss: %.3f, Accuracy: %.3f' %
            (epoch + 1, running_loss, 100 * correct / total))

    print('Finished Training')
    if saveModel: 
        torch.save(model, modelPathFile)
        
#load the model when doing only testing
else: 
    model = torch.load(modelPathFile)

#model evaluation
model.eval()

TypeError: compute_score_with_logits() missing 1 required positional argument: 'device'

In [ ]:
#if useCalcModel:
#    model = torch.load('model.pth')
correct = 0
total = 0
useImLevels = eval(os.getenv("useImLevels"))
perfect8 = 0
sevenLev = 0
problem = 0

modelPath = os.getenv('SAVE_LOAD_MODEL_PATH')
downImageSize = os.getenv('downImageSize')

logFname = "log" + str(nClasses) + "class" + str(downImageSize) + ".csv"
logPathFile = modelPath + logFname

logFile = open(logPathFile, "a")
titleString = "ProblemType ImageName C# FitScore "
logFile.write(titleString + "\n")

with torch.no_grad():
    for i, (images, labels, image_names) in enumerate(testloader, 0):
        if device == "mps":
            images = images.to(device)
            labels = tile(labels, 0, useImLevels).to(device)
        elif device == "cuda":
            images = images.cuda()
            labels = tile(labels, 0, useImLevels).cuda() 
        else:
            labels = tile(labels, 0, useImLevels)
        n_batches, n_crops, channels, height, width = images.size()
        image_batch = torch.autograd.Variable(images.view(-1, channels, height, width))
      
        outputs = model(image_batch)
        prob = outputs.cpu().numpy()
        prob = np.round(100*prob)/100
      
        nCl = len(prob[0])
        #initialize probability scorer
        fitN = np.zeros(nCl)
        #find maximum probability index 
        mpi =np.argmax(prob, axis=-1) #8-element vector w highest prob

        for j in range(nCl):
             fitN[j] = sum(np.equal(mpi, j))
        

        labeled_as = np.argmax(labels[0].cpu().numpy())
        logString =  image_names[0][0].split("_1_")[0] + " " + str(labeled_as+1) + " " + str(fitN.astype(int))

        #logFile.write(logString + "\n")

        # Print all images
        print("#", i+1,"Name:", image_names[0][0].split("_1_")[0], "Class #:", labeled_as+1, "Prob fit:", fitN.astype(int) )

        # Log only problematic Images:
        if max(fitN) == 8:
            perfect8 = perfect8 + 1
        elif max(fitN) == 7:
            sevenLev = sevenLev + 1
        else: # Max fit < 7
            problem = problem + 1

        #if (labeled_as == nCl-1) and (fitN[nCl-1]< 7):
            logFile.write('Doctor to Look' + logString + "\n")
            #print("#", i+1,"Name:", image_names[0][0].split("_1_")[0], "Class #:", labeled_as+1, "Prob fit:", fitN.astype(int) )
        #elif (labeled_as < nCl-1) and (fitN[nCl-1] > 4):
          #  problem = 'Sick as Normal '
          #  logFile.write(problem + logString + "\n")
            #Indicator of the fact that at least 4 Normal levels in sick image 
         

        fitScore = compute_score_with_logits(outputs, labels, device).sum()
        correct += fitScore.item()
        total += labels.size(0)

logFile.close()

print('Total images tested: ', len(testloader))
print('Accuracy according to Standard Metric: %.3f' % (100 * correct / total))
print('Images with all 8s: ', perfect8, ' with 7s: ', sevenLev) 
print('Accurate classification with 99.8% Conf,', 100*perfect8/len(testloader))
print('Accurate classification with at least 90% Conf,', 100*(perfect8+sevenLev)/len(testloader))